In [1]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import asyncio
import importlib
import pprint
from pathlib import Path
from typing import Any, Callable, Coroutine, Optional, Union

import aiohttp
import pandas as pd
# from aiohttp import ClientError, ClientTimeout
from bs4 import BeautifulSoup, NavigableString, ResultSet, Tag
# from chompjs import parse_js_object, parse_js_objects
from pandas import DataFrame
from selenium.webdriver.remote.webdriver import WebDriver
from selenium.webdriver.remote.webelement import WebElement
from sqlalchemy import inspect
import json
from pathlib import Path
from typing import Union
import json
from pathlib import Path
from bs4 import BeautifulSoup
import requests


def read_from_file(filepath: Path, parser: str) -> BeautifulSoup:
    with filepath.open(mode="r", encoding="utf-8") as file:
        return BeautifulSoup(file.read(), features=parser)


def write_to_file(filepath: Path, data: Union[str, dict], data_format: str = "text"):
    try:
        # Create a Path object from the provided file path
        path = Path(filepath)

        # Create parent directories if they don't exist
        path.parent.mkdir(parents=True, exist_ok=True)

        if data_format == "text":
            # If data is BeautifulSoup, convert it to a prettified string
            if isinstance(data, BeautifulSoup):
                data = str(data.prettify())

            # Write the content to the file as text
            with path.open(mode="w", encoding="utf-8") as file:
                file.write(data)
        elif data_format == "json":
            # If data is a dictionary, convert it to a JSON-formatted string
            if isinstance(data, (dict, list)):
                data = json.dumps(data, indent=4)

            # Write the JSON content to the file
            with path.open(mode="w", encoding="utf-8") as file:
                file.write(data)
        elif data_format == "md":
            with path.open(mode="w", encoding="utf-8") as file:
                file.write(data)            
        else:
            raise ValueError("Invalid data format. Use 'text' or 'json'.")

        print(f"Successfully wrote the data to '{filepath}' in {data_format} format.")
    except Exception as e:
        print(f"An error occurred: {e}")


def soup(
    source: Union[
        BeautifulSoup, NavigableString, ResultSet, Tag, WebElement, WebDriver, str, Path
    ],
    parser="html.parser",
    filepath: Optional[Path | str] = "temp.html",
) -> BeautifulSoup:
    soupen = None

    if isinstance(source, (BeautifulSoup, NavigableString, ResultSet, Tag)):
        soupen = source
    elif isinstance(source, Path) and source.exists():
        soupen = read_from_file(source, parser)
    elif isinstance(source, str):
        path = Path(source)
        if path.exists():
            soupen = read_from_file(path, parser)
        else:
            soupen = BeautifulSoup(source, features=parser)
    elif isinstance(source, WebElement):
        source_html: str = source.get_attribute("innerHTML")
        soupen = BeautifulSoup(source_html, features=parser)
    elif isinstance(source, WebDriver):
        source_html: str = source.page_source
        soupen = BeautifulSoup(source_html, features=parser)

    if filepath and soupen is not None:
        path = Path(filepath)
        path.parent.mkdir(parents=True, exist_ok=True)
        write_to_file(path, soupen)

    return soupen

In [2]:
async def _make_request(
    session: aiohttp.ClientSession,
    url: str,
    headers: Optional[dict] = None,
    cookies: Optional[dict] = None,
) -> Any:
    method = (
        headers.get("method").lower()
        if isinstance(headers.get("method"), str)
        else None
    )
    SessionChoice = getattr(session, method) if method else session
    async with SessionChoice(url, headers=headers, cookies=cookies) as response:
        if response.status == 200:
            # Check the content type of the response
            content_type = response.headers.get("Content-Type", "")
            # print(f"{content_type=}")
            if "application/json" in content_type:
                # If it's JSON, use .json() method
                return dict(data=await response.json(), status=response.status)
            else:
                # Otherwise, return as text
                return dict(data=await response.text(), status=response.status)
        print(f"Failed to fetch {url}. Status: {response.status}")
        return None


async def fetch(
    session: aiohttp.ClientSession,
    url: str,
    retries=3,
    timeout_for_wait: Optional[float] = None,
    headers: Optional[dict] = None,
    cookies: Optional[dict] = None,
) -> Any:
    for i in range(retries):
        try:
            return await asyncio.wait_for(
                _make_request(session, url, headers, cookies),
                timeout=timeout_for_wait,
            )
        except (aiohttp.ClientError, asyncio.TimeoutError) as e:
            if i == retries - 1:  # This was the last attempt
                print(f"Failed to fetch {url} after {retries} attempts. Error: {e}")
                return None


async def fetch_all(items, retries=3, timeout_for_session=5) -> list:
    timeout = aiohttp.ClientTimeout(total=timeout_for_session)
    async with aiohttp.ClientSession(timeout=timeout) as session:
        tasks = [
            fetch(
                session,
                item["url"],
                retries,
                timeout_for_session,
                item.get("headers"),
                item.get("cookies"),
            )
            for item in items
        ]
        return await asyncio.gather(*tasks)

In [1]:
from interface.backend import db

await db.init()

2023-09-13 11:40:17 INFO interface.backend.db - init - 28: Db initialized.


In [3]:
db.models.jobsch.Job.__tablename__

'jobsch_jobs'

In [3]:
BASE_URL: str = "https://www.jobs.ch/en/"
BASE_URL_JOBS: str = "https://www.jobs.ch/en/vacancies/"
BASE_URL_COMPANY: str = "https://www.jobs.ch/en/companies/"
BASE_URL_JOB_VAR: str = f"{BASE_URL_JOBS}" + "detail/{0}"
BASE_URL_COMPANY_VAR: str = "https://www.jobs.ch/en/companies/{0}/"
BASE_URL_JOBS_SEARCH: str = (
    BASE_URL_JOBS + "?location={location}&publication-date={pub_date}&term={term}"
)

BASE_API_URL_COMPANY_VAR: str = "https://www.jobs.ch/api/v1/public/company/{company_id}"

In [4]:
# rows=20
BASE_API_URL_JOBS_VAR = "https://www.jobs.ch/api/v1/public/search?location={location}&query={query}&rows={rows}"
referer = "https://www.jobs.ch/en/vacancies/?location=switzerland&term=python"


In [5]:
BASE_URL_JOB_VAR.format(
    "7293f5a4-7b9c-4b36-bd92-829c260260d3-electrical-engineer-80-100"
)


'https://www.jobs.ch/en/vacancies/detail/7293f5a4-7b9c-4b36-bd92-829c260260d3-electrical-engineer-80-100'

#### domain


In [4]:
# this is for the domain approach
async def extract_query_info(job_query: dict) -> dict[str, Any]:
    return {
        "clientClassification": job_query.get("botDetect", {}).get(
            "clientClassification"
        ),
        "pageTitle": job_query.get("seo", {}).get("meta", {}).get("pageTitle"),
        "pageDescription": job_query.get("seo", {})
        .get("meta", {})
        .get("pageDescription"),
        "pageList": str(
            job_query.get("vacancy", {}).get("traversal", {}).get("pageList")
        ),
        "numPages": job_query.get("vacancy", {})
        .get("results", {})
        .get("main", {})
        .get("meta", {})
        .get("numPages"),
        "totalHits": job_query.get("vacancy", {})
        .get("results", {})
        .get("main", {})
        .get("meta", {})
        .get("totalHits"),
        "searchQuery": job_query.get("vacancy", {})
        .get("results", {})
        .get("main", {})
        .get("meta", {})
        .get("searchQuery"),
    }


# queryInfo = extract_query_info(data_parsed[1])
# queryInfo['pageList'] = str(queryInfo['pageList'])
# query_df = pd.DataFrame(query_info).rename_axis('id')
# query_df.index += 1
# query_df
# queryInfo


async def handle_result(pkey, results, model, filepath):
    data_parsed = await analyse_html(results[0], filepath)

    queryInfo = await extract_query_info(data_parsed[1])

    await update_query([pkey], [queryInfo], model)

    return queryInfo, data_parsed

In [7]:
# query_df.to_sql('query', db.engineSync, if_exists='append')


In [ ]:
data_parsed[1]["botDetect"]["clientClassification"]
data_parsed[1]["seo"]["meta"]["pageTitle"]
data_parsed[1]["seo"]["meta"]["pageDescription"]
data_parsed[1]["vacancy"]["traversal"]["pageList"]
data_parsed[1]["vacancy"]["results"]["main"]["meta"]["numPages"]
data_parsed[1]["vacancy"]["results"]["main"]["meta"]["totalHits"]
data_parsed[1]["vacancy"]["results"]["main"]["meta"]["searchQuery"]


'bot'

'585 jobs in Switzerland - jobs.ch'

'Apply now for jobs in Switzerland! We have everything for your next job.'

[1]

30

585

'location=Switzerland'

### subqueries


In [8]:
def generate_subqueries_urls(base_url, numPages) -> list[str]:
    return [f"{base_url}&page={x}" for x in range(1, numPages + 1)]


async def generate_subqueries(queryInfo, queryPrimaryKey):
    numPages = queryInfo["numPages"]
    base_url = (await db.retrieve_objs([queryPrimaryKey], db.models.Query))[0].url
    if numPages > 1:
        urls: list[str] = generate_subqueries_urls(base_url, numPages)
        primaryIds: list[Any] = await db.create_objs(
            db.models.SubQuery, [dict(url=url, queryId=queryPrimaryKey) for url in urls]
        )
    else:
        primaryIds = [queryPrimaryKey]
    return primaryIds

In [9]:
async def mainEn(
    term="",
    location="Switzerland",
    pub_date="",
    headers: Optional[dict] = None,
    cookies: Optional[dict] = None,
) -> Coroutine[Any, Any, None]:
    queryPrimaryKey = (await create_query(term, location, pub_date))[0][0]

    items: list[dict[str, Any]] = [
        {
            "url": (await db.retrieve_objs([queryPrimaryKey], db.models.Query))[0].url,
            "name": "",
            "headers": {},
            "cookies": {},
            "function": None,  # This is a simple example function
        },
        # Add more dictionaries for more URLs
    ]

    results = await fetch_all(items)

    if results:
        queryInfo, data_parsed = await handle_result(
            queryPrimaryKey,
            results,
            db.models.Query,
            f"./data/html/query/{queryPrimaryKey}.html",
        )

        subQueriesPrimaryKeys: list[int] = await generate_subqueries(
            queryInfo, queryPrimaryKey
        )

        # scrape each subQueriesPrimaryKeys
        # fetch each subquery
        # extract subQueryInfo
        # extract Jobs(create jobs table with job+info, queryPm, subqueryPm)
        # increment jobs in in subquery and query

    return queryInfo, data_parsed, subQueriesPrimaryKeys
    # return queryPrimaryKey, queryInfo, subQueriesPrimaryKeys


queryInfo, data_parsed, subQueriesPrimaryKeys = await mainEn()

# queryPrimaryKey, queryInfo, subqueriesPrimaryIds = await mainEn()

In [10]:
subQueries = await db.retrieve_objs(subQueriesPrimaryKeys, db.models.SubQuery)
for idx, subq in enumerate(subQueries, start=1):
    items: list[dict[str, Any]] = [
        {
            "url": subq.url,
            "name": "",
            "headers": {},
            "cookies": {},
            "function": None,  # This is a simple example function
        },
        # Add more dictionaries for more URLs
    ]
    if result := await fetch_all(items):
        queryInfo: dict[str, Any] = await handle_result(
            subq.id,
            result,
            db.models.SubQuery,
            f"./data/html/subqueries/{subq.queryId}/{subq.id}.html",
        )

    await asyncio.sleep(1)
    if idx == 2:
        break

In [ ]:
JOB_LISTING_COLS: list[str] = [
    "title",
    "id",
    "initialPublicationDate",
    "publicationDate",
    "employmentGrades",
    "preview",
    "logo",
    "isActive",
    "languageSkills",
    "place",
    "salaryFrom",
    "salaryTo",
    "company.name",
    "company.id",
    "company.slug",
    "company.visible",
    "company.logoImage.src",
    "company.logoImage.height",
    "company.logoImage.width",
    "company_url",
    "job_url",
]


def construnct_job_url(job_id) -> str:
    return BASE_URL_JOB_VAR.format(job_id)


def construnct_company_url(company_id) -> str:
    return BASE_URL_COMPANY_VAR.format(company_id)


def extract_job_info(job) -> dict[str, Any]:
    return {
        "title": job.get("title", ""),
        "id": job.get("id", ""),
        "initialPublicationDate": job.get("initialPublicationDate", ""),
        "publicationDate": job.get("publicationDate", ""),
        "employmentGrades": job.get("employmentGrades", ""),
        "preview": job.get("preview", ""),
        "logo": job.get("logo", ""),
        "isActive": job.get("isActive", ""),
        "languageSkills": job.get("languageSkills", ""),
        "place": job.get("place", ""),
        "salaryFrom": job.get("salaryFrom", ""),
        "salaryTo": job.get("salaryTo", ""),
        "company_name": job.get("company.name", ""),
        "company_id": job.get("company.id", ""),
        "company_slug": job.get("company.slug", ""),
        "company_visible": job.get("company.visible", ""),
        "company_logoImage_src": job.get("company.logoImage.src", ""),
        "company_logoImage_height": job.get("company.logoImage.height", ""),
        "company_logoImage_width": job.get("company.logoImage.width", ""),
        "company_url": construnct_company_url(job.get("company.id", "")),
        "job_url": construnct_job_url(job.get("id", "")),
    }


async def scrape_jobs(data_parsed):
    jobs_df: DataFrame = pd.json_normalize(
        data_parsed[1]["vacancy"]["results"]["main"]["results"]
    )
    jobs_df.loc[:, "company_url"] = jobs_df["company.slug"].agg(construnct_company_url)
    jobs_df.loc[:, "job_url"] = jobs_df["id"].agg(construnct_job_url)
    jobs_df.loc[:, "languageSkills"] = jobs_df.loc[:, "languageSkills"].astype(str)

    jobs_df["initialPublicationDate"] = pd.to_datetime(
        jobs_df["initialPublicationDate"], format="%Y-%m-%dT%H:%M:%S%z"
    )
    jobs_df["publicationDate"] = pd.to_datetime(
        jobs_df["publicationDate"], format="%Y-%m-%dT%H:%M:%S%z"
    )

    jobs: DataFrame = jobs_df.loc[:, JOB_LISTING_COLS].set_index("id")

    job_dicts = [extract_job_info(job[1]) for job in jobs.iterrows()]
    return await db.create_objs(db.models.Job, job_dicts)


await scrape_jobs(data_parsed)

In [ ]:
"2023-08-04T17:06:49+02:00"


### jobs


In [ ]:
data_parsed[1]["vacancy"]["results"]["main"]["results"]
# write_string_to_txt_file("data_parsed.py", data_parsed)
with open("jobs.py", "w") as file:
    # Pretty print the list and write it to the file
    pprint.pprint(_, stream=file)


In [15]:
jobs_df: DataFrame = pd.json_normalize(
    data_parsed[1]["vacancy"]["results"]["main"]["results"]
)
jobs_df.columns


Index(['applicationQuestions', 'benefits', 'categories', 'contactAddress',
       'contactDetails', 'contactPerson', 'contacts', 'employmentGrades',
       'employmentGradesList', 'employmentPositionIds', 'employmentTypeIds',
       'headhunterApplicationAllowed', 'id', 'initialPublicationDate',
       'isActive', 'isInsecureExternal', 'isWebVacancy', 'languageSkills',
       'layoutType', 'listingTags', 'locations', 'offerId', 'place', 'preview',
       'publicationDate', 'regions', 'salaryFrom', 'salaryTo',
       'sourceHostname', 'title', 'videos', 'regionId', 'displayType',
       'company.id', 'company.name', 'company.slug', 'company.visible', 'logo',
       'company.companyLogoFile', 'company.logoImage.src',
       'company.logoImage.height', 'company.logoImage.width', 'mood'],
      dtype='object')

In [25]:
job_listing_cols: list[str] = [
    "title",
    "id",
    "initialPublicationDate",
    "publicationDate",
    "employmentGrades",
    "preview",
    "logo",
    "isActive",
    "languageSkills",
    "place",
    "salaryFrom",
    "salaryTo",
    "company.name",
    "company.id",
    "company.slug",
    "company.visible",
    "company.logoImage.src",
    "company.logoImage.height",
    "company.logoImage.width",
    "company_url",
    "job_url",
]


def construnct_job_url(job_id) -> str:
    return BASE_URL_JOB_VAR.format(job_id)


def construnct_company_url(company_id) -> str:
    return BASE_URL_COMPANY_VAR.format(company_id)


jobs_df.loc[:, "company_url"] = jobs_df["company.slug"].agg(construnct_company_url)
jobs_df.loc[:, "job_url"] = jobs_df["id"].agg(construnct_job_url)
jobs_df.loc[:, "languageSkills"] = jobs_df.loc[:, "languageSkills"].astype(str)

In [ ]:
jobs: DataFrame = jobs_df.loc[:, job_listing_cols].set_index("id")
jobs.shape

(20, 20)

In [ ]:
# save the dataframe to a table named 'users'
jobs.drop(["languageSkills"], axis=1).to_sql("jobs2", engineSync, if_exists="append")

20

### job


In [23]:
def extract_job_info(job) -> dict[str, Any]:
    return {
        "title": job.get("title", ""),
        "id": job.get("id", ""),
        "initialPublicationDate": job.get("initialPublicationDate", ""),
        "publicationDate": job.get("publicationDate", ""),
        "employmentGrades": job.get("employmentGrades", ""),
        "preview": job.get("preview", ""),
        "logo": job.get("logo", ""),
        "isActive": job.get("isActive", ""),
        "languageSkills": job.get("languageSkills", ""),
        "place": job.get("place", ""),
        "salaryFrom": job.get("salaryFrom", ""),
        "salaryTo": job.get("salaryTo", ""),
        "company.name": job.get("company.name", ""),
        "company.id": job.get("company.id", ""),
        "company.slug": job.get("company.slug", ""),
        "company.visible": job.get("company.visible", ""),
        "company.logoImage.src": job.get("company.logoImage.src", ""),
        "company.logoImage.height": job.get("company.logoImage.height", ""),
        "company.logoImage.width": job.get("company.logoImage.width", ""),
        "company.url": construnct_company_url(job.get("company.id", "")),
        "url": construnct_job_url(job.get("id", "")),
    }


# extract_job_info(jobs_df.iloc[0])

In [ ]:
jobs_df.iloc[0]["title"]
jobs_df.iloc[0]["id"]
jobs_df.iloc[0]["initialPublicationDate"]
jobs_df.iloc[0]["publicationDate"]
jobs_df.iloc[0]["employmentGrades"]
jobs_df.iloc[0]["preview"]
jobs_df.iloc[0]["logo"]
jobs_df.iloc[0]["isActive"]
jobs_df.iloc[0]["languageSkills"]
jobs_df.iloc[0]["place"]
jobs_df.iloc[0]["salaryFrom"]
jobs_df.iloc[0]["salaryTo"]
jobs_df.iloc[0]["company.name"]
jobs_df.iloc[0]["company.id"]
jobs_df.iloc[0]["company.slug"]
jobs_df.iloc[0]["company.visible"]
jobs_df.iloc[0]["company.logoImage.src"]
jobs_df.iloc[0]["company.logoImage.height"]
jobs_df.iloc[0]["company.logoImage.width"]


'Summer Camps Junior Coaches - Various Activities'

'c031309d-69c4-4efc-bcc9-e66b8f3b80d3'

'2023-07-04T10:06:55+02:00'

'2023-08-03T01:58:57+02:00'

'80% – 100%'

...

'78430-ecolint'

True

nan

nan

nan

# Domain


In [ ]:
# this is only needed for the domain url not the api
async def analyse_html(html: str, filepath: str):
    html_soup: BeautifulSoup = soup(source=html, filepath=filepath)

    script_tag: Tag | NavigableString | None = html_soup.find(
        "script",
        string=lambda x: all(
            text_string in x
            for text_string in [
                "__INIT__",
                "pageTitle",
                "pageDescription",
                "numPages",
                "totalHits",
            ]
        )
        if x is not None
        else False,
    )

    if script_tag:
        data_parsed = [x for x in list(parse_js_objects(script_tag.string)) if x]
        return data_parsed
    else:
        print("Script tag was not found in the html.")


# debug
# write_string_to_txt_file("scrapers/jobsdotch/temp.txt", script_tag.string)


# debug
# write_string_to_txt_file("data_parsed.py", data_parsed)
# with open("scrapers/jobsdotch/data_parsed.py", "w") as file:
# Pretty print the list and write it to the file
# pprint.pprint(data_parsed, stream=file)

# Api


## Request


In [8]:
from scrapers.jobsdotch import QueryBuilder

In [4]:
async def handle_request(items, filepath):
    data_json_list = await fetch_all(items)
    if data_json := data_json_list[0]:
        write_to_file(filepath, data_json.get("data", {}), "json")
        return data_json


async def create_query(query, location, days, url, model):
    return await db.create_objs(
        model,
        [dict(query=query, location=location, days=days, url=url)],
    )


def generate_subqueries_urls(base_url, numPages) -> list[str]:
    return [f"{base_url}&page={x}" for x in range(1, numPages + 1)]


async def generate_subqueries(
    num_pages, queryPrimaryKey, query, location, days, model_retrieve, model_create
):
    base_url = (await db.retrieve_objs([queryPrimaryKey], model_retrieve))[0].url
    if num_pages > 1:
        urls: list[str] = generate_subqueries_urls(base_url, num_pages)
        return await db.create_objs(
            model_create,
            [
                dict(
                    url=url,
                    qpkey=queryPrimaryKey,
                    query=query,
                    location=location,
                    days=days,
                    page=page,
                )
                for page, url in enumerate(urls, start=1)
            ],
        )
    else:
        return [queryPrimaryKey]

In [5]:
async def extract_query_info(rsp: dict):
    return dict(
        num_pages=rsp.get("data", {}).get("num_pages", ""),
        current_page=rsp.get("data", {}).get("current_page", ""),
        total_hits=rsp.get("data", {}).get("total_hits", ""),
        actual_hits=len(rsp.get("data", {}).get("documents", [])),
        status=rsp.get("status", None),
    )


# len(rsp.get('documents',''))

# await extract_query_info(results[0])


In [6]:
async def extract_job_info(job: dict):
    return {
        "title": job.get("title", ""),
        "publication_date": job.get("publication_date", ""),
        "company_name": job.get("company_name", ""),
        "place": job.get("place", ""),
        "is_active": job.get("is_active", ""),
        "preview": job.get("preview", ""),
        "company_logo_file": job.get("company_logo_file", ""),
        "job_id": job.get("job_id", ""),
        "slug": job.get("slug", ""),
        "company_slug": job.get("company_slug", ""),
        "company_id": job.get("company_id", ""),
        "company_segmentation": job.get("company_segmentation", ""),
        "employment_position_ids": str(job.get("employment_position_ids", [])),
        "employment_grades": str(job.get("employment_grades", [])),
        "is_paid": job.get("is_paid", ""),
        "work_experience": str(job.get("work_experience", [])),
        "language_skills": str(job.get("language_skills", [])),
        "job_url_en": job.get("_links").get("detail_de").get("href", ""),
        "job_url_de": job.get("_links").get("detail_fr").get("href", ""),
        "job_url_fr": job.get("_links").get("detail_en").get("href", "")
        # "links":job.get('_links', []),
    }


In [9]:
# pd.json_normalize(results[0]['documents'][0]).T

In [9]:
QueryBuilder(
    query="python",
    location="Switzerland",
).url_api


'https://www.jobs.ch/api/v1/public/search?location=Switzerland&query=python'

In [33]:
# block duplicate jobs and count their number, add a duplicate col in jobs
async def mainEn(
    query="",
    location="Switzerland",
    days="",
    headers: Optional[dict] = None,
    cookies: Optional[dict] = None,
) -> Coroutine[Any, Any, None]:
    if headers is None:
        headers = {}
    if cookies is None:
        cookies = {}

    # add full parameters
    query_url = QueryBuilder(query="python", location="Switzerland", days="").url_api

    query_pk = (
        await create_query(query, location, days, query_url, db.models.Query_Api)
    )[0][0]

    items: list[dict[str, Any]] = [
        {
            "url": query_url,
            "name": "",
            "headers": headers,
            "cookies": cookies,
        },
        # Add more dictionaries for more URLs
    ]

    data_json = await handle_request(items, f"data/json/queries/{query_pk}.json")

    if data_json.get("data", {}):
        query_info = await extract_query_info(data_json)

        await db.update_objs([query_pk], db.models.Query_Api, [query_info])

        subqueries_pkeys = await generate_subqueries(
            query_info["num_pages"],
            query_pk,
            query,
            location,
            days,
            db.models.Query_Api,
            db.models.SubQuery_Api,
        )

        for spkey in subqueries_pkeys:
            spkey = spkey[0]
            sub_data_json = await handle_request(
                items, f"data/json/sub_queries/{query_pk}/{spkey}.json"
            )
            sub_query_info = await extract_query_info(sub_data_json)
            await db.update_objs([spkey], db.models.SubQuery_Api, [sub_query_info])

            for job in sub_data_json.get("data", {}).get("documents", {}):
                db.duplicate_finder()
                job_info = await extract_job_info(job)
                job_info["query_id"] = query_pk
                job_info["subquery_id"] = spkey
                await db.create_objs(db.models.Job_Api, [job_info])

                # add duplicate check

    return (
        query_pk,
        data_json,
        query_info,
        spkey,
        sub_data_json,
        sub_query_info,
    )


query_pk, data_json, query_info, spkey, sub_data_json, sub_query_info = await mainEn()

# queryPrimaryKey, queryInfo, subqueriesPrimaryIds = await mainEn()

Successfully wrote the data to 'data/json/queries/1.json' in json format.
Successfully wrote the data to 'data/json/sub_queries/1/1.json' in json format.
Successfully wrote the data to 'data/json/sub_queries/1/2.json' in json format.
